In [1]:
## Script to analyze and output data for visualization
import pandas as pd
import numpy as np 

from scipy import stats

In [2]:
# Read in Data Files
# Data From: https://github.com/jldbc/coffee-quality-database
Acoffee_df = pd.read_csv("coffee-quality-database/data/arabica_data_cleaned.csv")
Rcoffee_df = pd.read_csv("coffee-quality-database/data/robusta_data_cleaned.csv")

# Join the two DFs
coffee_df = Acoffee_df.append(Rcoffee_df)

# Keep Columns 
coffee_df = coffee_df[["Species", "Country.of.Origin", "Aroma", 
                       "Flavor", "Aftertaste", "Acidity", "Body", 
                       "Balance", "Cupper.Points", "altitude_mean_meters"]]

### Data Cleaning ### 
shape_before_removeNA = coffee_df.shape[0] 

# Drop any NA values in Altitude Column 
coffee_df.dropna(subset = ["altitude_mean_meters"], inplace = True)

## Remove Outliers ##
z_scores = stats.zscore(coffee_df["altitude_mean_meters"])
abs_z_scores = np.abs(z_scores)

# Filter Outliers
filtered_elevations = (abs_z_scores < 3)
coffee_df = coffee_df[filtered_elevations]

# Give user update
print("Records Removed: {}. The Data set went from {} to {}".format((shape_before_removeNA - coffee_df.shape[0]), shape_before_removeNA, coffee_df.shape[0]))

### Group the Altitude ### 
# Add column to categorize by altitude
coffee_df["group"] = pd.cut(x=coffee_df["altitude_mean_meters"], 
                                      bins = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500], 
                                      labels = ["0 - .5k", ".5k - 1k", "1k - 1.5k", "1.5k - 2k", "2k - 2.5k", "2.5k - 3k", "3k - 3.5k", "3.5k - 4k", "4k - 4.5k"])
# Create Total Score
coffee_df["Total Score"] = (coffee_df["Aroma"] + coffee_df["Flavor"] + coffee_df["Aftertaste"] + coffee_df["Acidity"] + coffee_df["Body"] + coffee_df["Balance"] + coffee_df["Cupper.Points"])/7

coffee_df.head(5)

Records Removed: 233. The Data set went from 1339 to 1106


,Species,Country.of.Origin,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Cupper.Points,altitude_mean_meters,group,Total Score
0,Arabica,Ethiopia,8.67,8.83,8.67,8.75,8.50,8.42,8.75,2075.0,2k - 2.5k,8.655714
1,Arabica,Ethiopia,8.75,8.67,8.50,8.58,8.42,8.42,8.58,2075.0,2k - 2.5k,8.560000
2,Arabica,Guatemala,8.42,8.50,8.42,8.42,8.33,8.42,9.25,1700.0,1.5k - 2k,8.537143
3,Arabica,Ethiopia,8.17,8.58,8.42,8.42,8.50,8.25,8.67,2000.0,1.5k - 2k,8.430000
4,Arabica,Ethiopia,8.25,8.50,8.25,8.50,8.42,8.33,8.58,2075.0,2k - 2.5k,8.404286


In [3]:
## Create Pivot of all values based on Elevation Grouping ## 
coffee_df_grouped = coffee_df.groupby("group", as_index = False).mean()

# Remove straight altitude 
coffee_df_grouped.pop("altitude_mean_meters")
    
# unpivot columns 
coffee_df_grouped = coffee_df_grouped.melt(id_vars = ['group'], var_name = "variable", value_name = "value")

# round values 
coffee_df_grouped["value"] = coffee_df_grouped["value"].round(1)


coffee_df_grouped.head(5)

,group,variable,value
0,0 - .5k,Aroma,7.5
1,.5k - 1k,Aroma,7.5
2,1k - 1.5k,Aroma,7.5
3,1.5k - 2k,Aroma,7.7
4,2k - 2.5k,Aroma,8.0


In [4]:
## Geographic Data ##
coffee_geo = coffee_df

# Average Score per Country
coffee_geo.pop("group")
coffee_geo.pop("altitude_mean_meters")
coffee_geo.pop("Species")

coffee_geo.head()

,Country.of.Origin,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Cupper.Points,Total Score
0,Ethiopia,8.67,8.83,8.67,8.75,8.50,8.42,8.75,8.655714
1,Ethiopia,8.75,8.67,8.50,8.58,8.42,8.42,8.58,8.560000
2,Guatemala,8.42,8.50,8.42,8.42,8.33,8.42,9.25,8.537143
3,Ethiopia,8.17,8.58,8.42,8.42,8.50,8.25,8.67,8.430000
4,Ethiopia,8.25,8.50,8.25,8.50,8.42,8.33,8.58,8.404286


In [6]:
# Drop any NA values in
coffee_df.dropna(inplace = True)

# Group by Country of Origin
coffee_geo_grouped = coffee_geo.groupby("Country.of.Origin", as_index = False).mean()

# Round Results 
coffee_geo_grouped["Total Score"] = coffee_geo_grouped["Total Score"].round(1)

coffee_geo_grouped = coffee_geo_grouped[["Country.of.Origin", "Total Score"]]

,Country.of.Origin,Total Score
0,Brazil,7.6
1,Burundi,7.4
2,China,7.6
3,Colombia,7.6
4,Costa Rica,7.6


In [7]:
## Export dfs ## 

# Output Total Dataset
coffee_df.to_csv("total_coffee.csv")

# Output the scored data set
coffee_df_grouped.to_csv("coffee_reviews.csv")

# Output the coffee geo scores
coffee_geo_grouped.to_csv("coffee_geo.csv")